# Introduction to MongoDB in Python
## 1. Flexbily Structured Data

In [2]:
import os
from pymongo import MongoClient
from operator import itemgetter
from collections import Counter

MONGODB_URI = os.getenv('MONGODB_URI')

client = MongoClient(MONGODB_URI)
db = client["nobel"]

In [2]:
filter = {}
print(db.prizes.count_documents(filter))
print(db.laureates.count_documents(filter))

652
955


In [3]:
db_names = client.list_database_names()
db_names

['admin', 'config', 'local', 'nobel']

In [4]:
nobel_coll_names = client.nobel.list_collection_names()
nobel_coll_names

['prizes', 'laureates']

In [5]:
prize = db.prizes.find_one()
laureate = db.laureates.find_one()

print(prize)
print(laureate)
print(type(laureate))

{'_id': ObjectId('5ffa97fa9d6167d9d146dd5d'), 'year': '2020', 'category': 'chemistry', 'laureates': [{'id': '991', 'firstname': 'Emmanuelle', 'surname': 'Charpentier', 'motivation': '"for the development of a method for genome editing"', 'share': '2'}, {'id': '992', 'firstname': 'Jennifer A.', 'surname': 'Doudna', 'motivation': '"for the development of a method for genome editing"', 'share': '2'}]}
{'_id': ObjectId('5ffa982f9d6167d9d146dfe9'), 'id': '1', 'firstname': 'Wilhelm Conrad', 'surname': 'Röntgen', 'born': '1845-03-27', 'died': '1923-02-10', 'bornCountry': 'Prussia (now Germany)', 'bornCountryCode': 'DE', 'bornCity': 'Lennep (now Remscheid)', 'diedCountry': 'Germany', 'diedCountryCode': 'DE', 'diedCity': 'Munich', 'gender': 'male', 'prizes': [{'year': '1901', 'category': 'physics', 'share': '1', 'motivation': '"in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him"', 'affiliations': [{'name': 'Munich Un

In [6]:
prize_fields = list(prize.keys())
laureate_fields = list(laureate.keys())

print(prize_fields)
print(laureate_fields)

['_id', 'year', 'category', 'laureates']
['_id', 'id', 'firstname', 'surname', 'born', 'died', 'bornCountry', 'bornCountryCode', 'bornCity', 'diedCountry', 'diedCountryCode', 'diedCity', 'gender', 'prizes']


## 2. Working with Distinct Values and Sets

In [7]:
set(db.prizes.distinct("category"))

{'chemistry', 'economics', 'literature', 'medicine', 'peace', 'physics'}

In [8]:
set(db.laureates.distinct("prizes.category"))

{'chemistry', 'economics', 'literature', 'medicine', 'peace', 'physics'}

In [9]:
set(db.prizes.distinct("category")) == set(db.laureates.distinct("prizes.category"))


True

In [10]:
countries = set(db.laureates.distinct("diedCountry")) - set(db.laureates.distinct("bornCountry"))
countries

{'Barbados',
 'East Germany (now Germany)',
 'Gabon',
 'Greece',
 'Israel',
 'Jamaica',
 'Northern Rhodesia (now Zambia)',
 'Philippines',
 'Puerto Rico',
 'Singapore',
 'Tunisia',
 'Yugoslavia (now Serbia)'}

In [11]:
db.laureates.find_one()

{'_id': ObjectId('5ffa982f9d6167d9d146dfe9'),
 'id': '1',
 'firstname': 'Wilhelm Conrad',
 'surname': 'Röntgen',
 'born': '1845-03-27',
 'died': '1923-02-10',
 'bornCountry': 'Prussia (now Germany)',
 'bornCountryCode': 'DE',
 'bornCity': 'Lennep (now Remscheid)',
 'diedCountry': 'Germany',
 'diedCountryCode': 'DE',
 'diedCity': 'Munich',
 'gender': 'male',
 'prizes': [{'year': '1901',
   'category': 'physics',
   'share': '1',
   'motivation': '"in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him"',
   'affiliations': [{'name': 'Munich University',
     'city': 'Munich',
     'country': 'Germany'}]}]}

In [12]:
set(db.laureates.distinct("prizes.affiliations.country"))

{'Argentina',
 'Australia',
 'Austria',
 'Belgium',
 'Canada',
 'China',
 'Czechoslovakia (now Czech Republic)',
 'Denmark',
 'Finland',
 'France',
 'Germany',
 'Germany (now France)',
 'Hungary',
 'India',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Norway',
 'Portugal',
 'Russia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Tunisia',
 'USA',
 'USSR (now Russia)',
 'United Kingdom',
 'the Netherlands'}

In [13]:
db.laureates.find_one()

{'_id': ObjectId('5ffa982f9d6167d9d146dfe9'),
 'id': '1',
 'firstname': 'Wilhelm Conrad',
 'surname': 'Röntgen',
 'born': '1845-03-27',
 'died': '1923-02-10',
 'bornCountry': 'Prussia (now Germany)',
 'bornCountryCode': 'DE',
 'bornCity': 'Lennep (now Remscheid)',
 'diedCountry': 'Germany',
 'diedCountryCode': 'DE',
 'diedCity': 'Munich',
 'gender': 'male',
 'prizes': [{'year': '1901',
   'category': 'physics',
   'share': '1',
   'motivation': '"in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him"',
   'affiliations': [{'name': 'Munich University',
     'city': 'Munich',
     'country': 'Germany'}]}]}

In [14]:
# In which countries have USA-born laureates had affiliations for their prizes?
db.laureates.distinct("prizes.affiliations.country", {"bornCountry": "USA"})

['Australia', 'Denmark', 'USA', 'United Kingdom']

In [15]:
# Confirm via an assertion that "literature" is the only prize category with no prizes shared by three or more laureates.
criteria = {"laureates.2": {"$exists": True}}
triple_play_categories = set(db.prizes.distinct("category", criteria))
assert set(['literature']) == set(db.prizes.distinct("category")) - triple_play_categories


In [16]:
# What is the approximate ratio of the number of laureates who won an unshared ({"share": "1"}) prize in physics after World War II ({"year": {"$gte": "1945"}}) to the number of laureates who won a shared prize in physics after World War II?
a = db.laureates.count_documents({"prizes": {"$elemMatch": {"share": "1", "category": "physics", "year": {"$gte": "1945"}}}})
b = db.laureates.count_documents({"prizes": {"$elemMatch": {"share": {"$ne": "1"}, "category": "physics", "year": {"$gte": "1945"}}}})
a/b

0.12080536912751678

In [17]:
# What is this ratio for prize categories other than physics, chemistry, and medicine?
unshared = {"prizes": {"$elemMatch": {"share": "1", "category": {"$nin": ["physics", "chemistry", "medicine"]}, "year": {"$gte": "1945"}}}}
shared = {"prizes": {"$elemMatch": {"share": {"$ne": "1"}, "category": {"$nin": ["physics", "chemistry", "medicine"]}, "year": {"$gte": "1945"}}}}
count_unshared = db.laureates.count_documents(unshared)
count_shared = db.laureates.count_documents(shared)
ratio = count_unshared / count_shared
ratio

1.348623853211009

In [18]:
before = {"gender": "org", "prizes.year": {"$lt": "1945"}}
in_or_after = {"gender": "org", "prizes.year": {"$gte": "1945"}}
count_before = db.laureates.count_documents(before)
count_in_or_after = db.laureates.count_documents(in_or_after)
ratio = count_in_or_after / (count_before + count_in_or_after)
ratio

0.8461538461538461

In [19]:
# How many laureates in total have a first name beginning with "G" and a surname beginning with "S"?
from bson.regex import Regex
db.laureates.count_documents({"firstname": Regex("^G"), "surname": Regex("^S")})

10

In [20]:
# Use a regular expression object to filter for laureates with "Germany" in their "bornCountry" value.
criteria = {"bornCountry": Regex("Germany")}
set(db.laureates.distinct("bornCountry", criteria))

{'Bavaria (now Germany)',
 'East Friesland (now Germany)',
 'Germany',
 'Germany (now France)',
 'Germany (now Poland)',
 'Germany (now Russia)',
 'Hesse-Kassel (now Germany)',
 'Mecklenburg (now Germany)',
 'Prussia (now Germany)',
 'Schleswig (now Germany)',
 'West Germany (now Germany)',
 'Württemberg (now Germany)'}

In [21]:
# Use a regular expression object to filter for laureates with a "bornCountry" value starting with "Germany".
criteria = {"bornCountry": Regex("^Germany")}
set(db.laureates.distinct("bornCountry", criteria))

{'Germany',
 'Germany (now France)',
 'Germany (now Poland)',
 'Germany (now Russia)'}

In [22]:
# Use a regular expression object to filter for laureates born in what was at the time Germany but is now another country.
criteria = {"bornCountry": Regex("^Germany \(now")}
set(db.laureates.distinct("bornCountry", criteria))

{'Germany (now France)', 'Germany (now Poland)', 'Germany (now Russia)'}

In [23]:
# Use a regular expression object to filter for laureates born in what is now Germany but at the time was another country.
criteria = {"bornCountry": Regex("now Germany\)$")}
set(db.laureates.distinct("bornCountry", criteria))

{'Bavaria (now Germany)',
 'East Friesland (now Germany)',
 'Hesse-Kassel (now Germany)',
 'Mecklenburg (now Germany)',
 'Prussia (now Germany)',
 'Schleswig (now Germany)',
 'West Germany (now Germany)',
 'Württemberg (now Germany)'}

In [24]:
# We can filter on "transistor" as a substring of a laureate's "prizes.motivation" field value to find these laureates.
criteria = {"prizes.motivation": Regex("transistor")}
first = "firstname"
last = "surname"
print([(laureate[first], laureate[last]) for laureate in db.laureates.find(criteria)])

[('William B.', 'Shockley'), ('John', 'Bardeen'), ('Walter H.', 'Brattain')]


## 3. Get Only What You Need, and Fast

In [25]:
db.laureates.find_one()

{'_id': ObjectId('5ffa982f9d6167d9d146dfe9'),
 'id': '1',
 'firstname': 'Wilhelm Conrad',
 'surname': 'Röntgen',
 'born': '1845-03-27',
 'died': '1923-02-10',
 'bornCountry': 'Prussia (now Germany)',
 'bornCountryCode': 'DE',
 'bornCity': 'Lennep (now Remscheid)',
 'diedCountry': 'Germany',
 'diedCountryCode': 'DE',
 'diedCity': 'Munich',
 'gender': 'male',
 'prizes': [{'year': '1901',
   'category': 'physics',
   'share': '1',
   'motivation': '"in recognition of the extraordinary services he has rendered by the discovery of the remarkable rays subsequently named after him"',
   'affiliations': [{'name': 'Munich University',
     'city': 'Munich',
     'country': 'Germany'}]}]}

In [26]:
# fetch ONLY the laureates' full names and prize share info?
db.laureates.find_one(filter={"prizes": {"$elemMatch": {"category": "physics", "year": "1903"}}}, projection={"firstname": 1, "surname":1, "prizes.share":1, "_id":0})


{'firstname': 'Henri', 'surname': 'Becquerel', 'prizes': [{'share': '2'}]}

In [27]:
docs = db.laureates.find(filter={"firstname": {"$regex": "^G"}, "surname": {"$regex": "^S"}}, projection={"firstname": 1, "surname": 1, "_id": 0})
full_name_list = [doc["firstname"]+" "+ doc["surname"] for doc in docs]
full_name_list

['Glenn T. Seaborg',
 'George D. Snell',
 'Gustav Stresemann',
 'George Bernard Shaw',
 'Giorgos Seferis',
 'George J. Stigler',
 'George F. Smoot',
 'George E. Smith',
 'George P. Smith',
 'Gregg Semenza']

In [53]:
# check that for each prize, all the shares of all the laureates add up to 1
docs = db.prizes.find(filter={"laureates.0": {"$exists": True}}, projection={"laureates.share": 1, "_id": 0})
for doc in docs:
    each = doc['laureates']
    sum = 0
    for i in range(len(each)):
        share = int(each[i]["share"])
        sum += 1 / share
    print(sum)

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [57]:
docs = list(db.laureates.find({"born": {"$gte": "1900"}, "prizes.year": {"$gte": "1954"}}, {"born": 1, "prizes.year":1, "_id":0}, sort=[("prizes.year", 1), ("born", -1)]))
for doc in docs[:5]:
    print(doc)

{'born': '1950-12-14', 'prizes': [{'year': '1954'}, {'year': '1981'}]}
{'born': '1916-08-25', 'prizes': [{'year': '1954'}]}
{'born': '1915-06-15', 'prizes': [{'year': '1954'}]}
{'born': '1901-02-28', 'prizes': [{'year': '1962'}, {'year': '1954'}]}
{'born': '1913-07-12', 'prizes': [{'year': '1955'}]}


In [66]:
sample_prize = db.prizes.find_one()

def all_laureates(prize):
    sorted_laureates = sorted(prize["laureates"], key=itemgetter("surname"))

    surnames = [laureate["surname"] for laureate in sorted_laureates]

    all_names = " and ".join(surnames)

    return all_names

print(all_laureates(sample_prize))

Charpentier and Doudna


In [74]:
docs = list(db.prizes.find(filter={"category": "physics", "laureates": {"$exists": True}}, projection={"year": 1, "laureates.firstname": 1, "laureates.surname": 1, "_id": 0}, sort=[("year", 1)]))

In [78]:
for doc in docs:
    print(doc["year"], all_laureates(doc))

1901 Röntgen
1902 Lorentz and Zeeman
1903 Becquerel and Curie and Curie
1904 Rayleigh
1905 Lenard
1906 Thomson
1907 Michelson
1908 Lippmann
1909 Braun and Marconi
1910 van der Waals
1911 Wien
1912 Dalén
1913 Kamerlingh Onnes
1914 von Laue
1915 Bragg and Bragg
1917 Barkla
1918 Planck
1919 Stark
1920 Guillaume
1921 Einstein
1922 Bohr
1923 Millikan
1924 Siegbahn
1925 Franck and Hertz
1926 Perrin
1927 Compton and Wilson
1928 Richardson
1929 de Broglie
1930 Raman
1932 Heisenberg
1933 Dirac and Schrödinger
1935 Chadwick
1936 Anderson and Hess
1937 Davisson and Thomson
1938 Fermi
1939 Lawrence
1943 Stern
1944 Rabi
1945 Pauli
1946 Bridgman
1947 Appleton
1948 Blackett
1949 Yukawa
1950 Powell
1951 Cockcroft and Walton
1952 Bloch and Purcell
1953 Zernike
1954 Born and Bothe
1955 Kusch and Lamb
1956 Bardeen and Brattain and Shockley
1957 Lee and Yang
1958 Cherenkov and Frank and Tamm
1959 Chamberlain and Segrè
1960 Glaser
1961 Hofstadter and Mössbauer
1962 Landau
1963 Goeppert Mayer and Jensen and

In [79]:
criteria = {"year": "1901"}
original_prize = db.prizes.distinct("category", criteria)

['chemistry', 'literature', 'medicine', 'peace', 'physics']

In [90]:
docs = list(db.prizes.find(projection={"year": 1, "category": 1, "_id": 0}, sort=[("year", -1), ("category", 1)]))
docs[:10]

[{'year': '2020', 'category': 'chemistry'},
 {'year': '2020', 'category': 'economics'},
 {'year': '2020', 'category': 'literature'},
 {'year': '2020', 'category': 'medicine'},
 {'year': '2020', 'category': 'peace'},
 {'year': '2020', 'category': 'physics'},
 {'year': '2019', 'category': 'chemistry'},
 {'year': '2019', 'category': 'economics'},
 {'year': '2019', 'category': 'literature'},
 {'year': '2019', 'category': 'medicine'}]

In [92]:
%%timeit -r2 -n10
a+b     

200 ns ± 80 ns per loop (mean ± std. dev. of 2 runs, 10 loops each)


In [11]:
db.prizes.distinct("category", {"laureates.share": {"$gte": "3"}})

['chemistry', 'economics', 'medicine', 'peace', 'physics']

In [54]:
import sys
# For each prize category, report the most recent year that a single laureate -- rather than several -- received a prize in that category.
index_model = [("category", 1), ("year", -1)]
db.prizes.create_index(index_model)

report=""
for category in sorted(db.prizes.distinct("category")):
    result = db.prizes.find_one(filter={"category": category, "laureates.share": "1"}, projection={"category":1, "year": 1, "_id": 0}, sort=[("sort", -1)])
    report += "{category}: {year}\n".format(**result)

print(report)
    

chemistry: 2011
economics: 2017
literature: 2020
medicine: 2016
peace: 2020
physics: 1992



In [75]:
# Some countries are, for one or more laureates, both their country of birth ("bornCountry") and a country of affiliation for one or more of their prizes ("prizes.affiliations.country"). You will find the five countries of birth with the highest counts of such laureates.
from collections import Counter

db.laureates.create_index([("bornCountry", 1)])

country_count = {country: db.laureates.count_documents({"bornCountry": country, "prizes.affiliations.country": country}) for country in db.laureates.distinct("bornCountry")}

top5 = Counter(country_count).most_common(5)
print(top5)



# My initial answer(very wrong)
# countries_list = db.laureates.distinct(key="bornCountry")
# countries_count = {}
# for country in countries_list:
#     countries_count[country] = 0

# for laureate in db.laureates.find(filter={"bornCountry": {"$exists": True}}):
#     if db.laureates.count_documents({"prizes.affiliations.country": laureate["bornCountry"]}) >= 1:  <-------Terribly logically wrong
#         countries_count[laureate["bornCountry"]] += 1

# list_of_set = []
# list(sorted(countries_count.items(), key=lambda item: item[1], reverse=True))[:5]

[('USA', 248), ('United Kingdom', 58), ('Germany', 37), ('France', 26), ('Japan', 18)]


In [83]:
# You can think of the query parameters as being updated like a dictionary in Python
list(db.prizes.find({"category": "economics"}, {"year": 1, "_id": 0}).sort("year").limit(3).limit(5))

[{'year': '1969'},
 {'year': '1970'},
 {'year': '1971'},
 {'year': '1972'},
 {'year': '1973'}]

In [90]:
list(db.prizes.find(filter={"laureates.share": "4"}, projection={"category": "1", "year": "1", "laureates.motivation": "1"}).sort("year").limit(5))

[{'_id': ObjectId('5ffa97fa9d6167d9d146dfdd'),
  'year': '1903',
  'category': 'physics',
  'laureates': [{'motivation': '"in recognition of the extraordinary services he has rendered by his discovery of spontaneous radioactivity"'},
   {'motivation': '"in recognition of the extraordinary services they have rendered by their joint researches on the radiation phenomena discovered by Professor Henri Becquerel"'},
   {'motivation': '"in recognition of the extraordinary services they have rendered by their joint researches on the radiation phenomena discovered by Professor Henri Becquerel"'}]},
 {'_id': ObjectId('5ffa97fa9d6167d9d146df03'),
  'year': '1946',
  'category': 'chemistry',
  'laureates': [{'motivation': '"for his discovery that enzymes can be crystallized"'},
   {'motivation': '"for their preparation of enzymes and virus proteins in a pure form"'},
   {'motivation': '"for their preparation of enzymes and virus proteins in a pure form"'}]},
 {'_id': ObjectId('5ffa97fa9d6167d9d14

In [98]:
from pprint import pprint

page_number = 1
page_skip = 9 * (page_number - 1)
page_size = 9

def get_particle_laureates(page_number=1, page_size=3):
    if page_number < 1 or not isinstance(page_number, int):
        raise ValueError("Pages are natural numbers (starting from 1).")
    page_result = list(db.laureates.find(filter={"prizes.motivation": {"$regex": "particle"}}).sort([("prizes.year", 1), ("surname", 1)]).skip(page_size * (page_number - 1)).limit(page_size))
    return page_result

pages = [get_particle_laureates(page_number=page) for page in range(1,10)]
pprint(pages[0])

[{'_id': ObjectId('5ffa982f9d6167d9d146e009'),
  'born': '1869-02-14',
  'bornCity': 'Glencorse',
  'bornCountry': 'Scotland',
  'bornCountryCode': 'GB',
  'died': '1959-11-15',
  'diedCity': 'Carlops',
  'diedCountry': 'Scotland',
  'diedCountryCode': 'GB',
  'firstname': 'C.T.R.',
  'gender': 'male',
  'id': '34',
  'prizes': [{'affiliations': [{'city': 'Cambridge',
                                'country': 'United Kingdom',
                                'name': 'University of Cambridge'}],
              'category': 'physics',
              'motivation': '"for his method of making the paths of '
                            'electrically charged particles visible by '
                            'condensation of vapour"',
              'share': '2',
              'year': '1927'}],
  'surname': 'Wilson'},
 {'_id': ObjectId('5ffa982f9d6167d9d146e01f'),
  'born': '1897-05-27',
  'bornCity': 'Todmorden',
  'bornCountry': 'United Kingdom',
  'bornCountryCode': 'GB',
  'died': '1967-09-1